# Importamos las librerías requeridas

In [198]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import asyncio
import pandas as pd
import re

In [3]:
driver = webdriver.Chrome()

In [4]:
def iniciarSesion(name,password):
    driver.get('https://www.linkedin.com/') # Abrir linkedin
    time.sleep(5)

    username = driver.find_element(By.ID, "session_key")
    username.send_keys(name) # Poner correo 

    pword = driver.find_element(By.ID, "session_password")
    pword.send_keys(password)    #Poner tu contraseña
    time.sleep(5)  

    driver.find_element(By.XPATH, "//button[@type='submit']").click() # Tocar el botón de iniciar sesión 

In [6]:
iniciarSesion('####@gmail.com','#####')

In [7]:
def trabajo(url):
    driver.get(url) ## Abrir la urls de empleos. Por ejemplo: Analista de datos 
    time.sleep(3)
    # Filtrado por fecha
    wait = WebDriverWait(driver, 30)
    button = wait.until(EC.element_to_be_clickable((By.ID, "searchFilter_timePostedRange")))
    button.click()
    
    option = wait.until(EC.presence_of_element_located((By.ID, "timePostedRange-r86400")))
    ## Filtrar por 24 horas
    driver.execute_script("arguments[0].click();", option)

    # Click en mostrar los resultados del filtro
    driver.find_element(By.XPATH, "//button[@type='button']").click()

In [124]:
trabajo(f"https://www.linkedin.com/jobs/search/?currentJobId=3692544132&keywords=data%20science&refresh=true")

In [126]:
linksJobs = []

In [128]:
locationCompany = []

In [130]:
def obtencionLinkJobs(num_pages):
    try: 
        for page in range(2,num_pages+2):
            time.sleep(2)
            jobs = driver.find_element(By.CLASS_NAME ,'jobs-search-results-list')
            jobs_list= jobs.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')
        
            for job in jobs_list:
                all_links = job.find_elements(By.TAG_NAME, "a")
                for a in all_links:
                    if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in linksJobs: 
                        linksJobs.append(a.get_attribute('href')) # agregar los links en la lista
                        locationCompany.append(job.find_element(By.CLASS_NAME,"job-card-container__metadata-item ").text)
                    else:
                        pass
                # scroll 
                driver.execute_script("arguments[0].scrollIntoView();", job)

            
            print(f'Links cargados de la página: {page-1}') # imprimir la página a la cual ha extraídos los links

            driver.find_element(By.XPATH,f"//button[@aria-label='Página {page}']").click() # Pasar a la página siguiente
            time.sleep(3)
    except:
        pass

In [131]:
obtencionLinkJobs(5)

Links cargados de la página: 1
Links cargados de la página: 2
Links cargados de la página: 3
Links cargados de la página: 4
Links cargados de la página: 5


In [132]:
len(linksJobs)

103

In [133]:
len(locationCompany)

103

In [134]:
linksJobs

['https://www.linkedin.com/jobs/view/3723261581/?eBP=CwEAAAGKs9ytnvVmfuPsJ24oVGsHRQVBCYvrnq0nI0MmqyAcPwhG8EfbMz0P57CAuwwHAAgMY7S8mMUP1hPuw3YvZyNMZfF0T52kaWNlzjkj-usw1E32Xp9xmFeZJIDI6BYpjjCH3zIAnwaoA2wrpdfl2D0fNYTFMUc7V8OcANUvd3kHhuEdZ65WeIwxPBaUU74rmv0bv6OfLGTMtQhuDI7YigSK0ONTuifQJlc-bkKpc1wu4sq9vMVk9IQYs8H-JBkge5G95uTAbjrNT7wO5Ks-Viv2S0wIh8vGx2Lck5Y0tJT5twTFnmz-4joXSvQRJPTq4bVtt05il2xXEj7r2epX83144t6VROD97Sz2VQtWOnSXciNLxQn1e4ShlDNhOfm8wSIIb9ZVwji0y7iMf5AJXrWt1NkFObyizftN4XQ&refId=zCefCQABH%2FgQJWHGyqSVfg%3D%3D&trackingId=BKtpzRWycj%2BFZg3DvUUNDw%3D%3D&trk=flagship3_search_srp_jobs',
 'https://www.linkedin.com/jobs/view/3723215447/?eBP=CwEAAAGKs9ytnlSYYSARvl6kyWnh-2ROfNUL0c8Kx1p4PlY_a6eC78xcP9fU--30a_gOmTrpGPTzvXvQxxRFgGHE6mhETh4mn5wwdrI_DOe7PR9fja-Crt3ZA0IeNmrr52mymXWczynJOxs0YNHzVP_gQbEIFo_u4UjCIsHVJTfXwYNDjueDTSnMH0enXlXWVA9FdR4PjZBXwh5gAf6lvKgi-TapoBaAB1pz54q1qgHRj0awPv28LBSAD3jO-gvq-zv7EfdeHUD0gYu2ebGeO9fZsDRiDjx6IccD1cb8SgGjR-S1f9ER5lpjzoW_OJDppUSsRrHc2JkmIuwlCA_9O9S4ax57-KGwJn5

In [147]:
titleJobs = []
companyName = []
employeeRank = []
dateJobs = []
workType = []

In [150]:
skill = [] 

In [152]:
def getDate(time):
    fecha_hora_actual = datetime.now()
    if time.split()[1] == 'horas' or time.split()[1] == 'hora':
        hora = time.split()[0]
        nueva_fecha = fecha_hora_actual - timedelta(hours=int(hora))
        fecha_formateada = nueva_fecha.strftime("%d/%m/%Y")
        return fecha_formateada
    if time.split()[1] == 'minutos' or time.split()[1] == 'minuto':
        minutos = time.split()[0]
        nueva_fecha = fecha_hora_actual - timedelta(minutes=int(minutos))
        fecha_formateada = nueva_fecha.strftime("%d/%m/%Y")
        return fecha_formateada
    if time.split()[1] == 'segundos' or time.split()[1] == 'segundo':
        segundos = time.split()[0]
        nueva_fecha = fecha_hora_actual - timedelta(seconds=int(segundos))
        fecha_formateada = nueva_fecha.strftime("%d/%m/%Y")
        return fecha_formateada
    else: return 'Dato no encontrado'

In [157]:
async def Extractdata():
    for i in range(len(linksJobs)):
        try:
            driver.get(linksJobs[i])
            time.sleep(2)
        except:
            pass
        
        # Filtración por la parte donde extraeremos los datos
        
        contents = driver.find_elements(By.CLASS_NAME,'p5')
        for content in contents:
      
            filtroComapany = content.find_element(By.CLASS_NAME,'jobs-unified-top-card__primary-description')
            titleJobs.append(content.find_element(By.TAG_NAME,"h1").text)
            try: companyName.append(filtroComapany.find_element(By.CLASS_NAME,'app-aware-link').text)
            except NoSuchElementException: companyName.append('Dato no encontrado')
            try: workType.append(driver.find_elements(By.CLASS_NAME,"jobs-unified-top-card__job-insight")[0].text)
            except NoSuchElementException: workType.append('Dato no encontrado')
            try: employeeRank.append(driver.find_elements(By.CLASS_NAME,"jobs-unified-top-card__job-insight")[1].text)
            except NoSuchElementException: employeeRank.append('Dato no encontrado')
            try: dateJobs.append(getDate(content.find_element(By.TAG_NAME,'strong').text.split('hace')[1].strip()))
            except NoSuchElementException: dateJobs.append('Dato no encontrado')

In [161]:
async def ExtractSkill():
    for i in range(len(linksJobs)):
        driver.get(linksJobs[i])
        time.sleep(2)
        try:
            fil = driver.find_element(By.CLASS_NAME,f"job-details-how-you-match-card__container")
            fil.find_element(By.TAG_NAME,'button').click()
            time.sleep(5)
            filtro = driver.find_elements(By.CLASS_NAME,"job-details-skill-match-status-list")[0].text.replace("\n", "/").split('/')
            lista1 = [filtro for filtro in filtro if filtro != "Añadir"]
            skill.append(lista1)
        except NoSuchElementException:
            skill.append('Dato no encontrado')

In [162]:
await ExtractSkill()

In [159]:
await Extractdata()

In [164]:
df = pd.DataFrame({'titleJobs': titleJobs,'comanyName': companyName,'companyLocation': locationCompany,'employeeRank': employeeRank,
'workType': workType, 'dateJobs': dateJobs,'skill': skill, 'link': linksJobs}) #Lo convertimos a dataframe

In [165]:
df

,titleJobs,comanyName,companyLocation,employeeRank,workType,dateJobs,skill,link
0,Mid. QA Automation Engineer - Buenos Aires,Devlane,Buenos Aires y alrededores (Híbrido),De 51 a 200 empleados,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3723261581/...
1,Data Scientist,Rocbird,"Córdoba, Córdoba, Argentina (Híbrido)",De 11 a 50 empleados,Jornada completa,20/09/2023,"[Análisis de datos, Ciencia de datos, Microsof...",https://www.linkedin.com/jobs/view/3723215447/...
2,Financial Analyst I - Pto. Madero,Worley,"Buenos Aires, Provincia de Buenos Aires, Argen...",Más de 10.001 empleados · Servicios profesionales,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3725055225/...
3,Jefe de Nuevas Obras y Proyectos de Infraestru...,Clínica Universitaria Reina Fabiola,"Córdoba, Córdoba, Argentina (Presencial)",De 1.001 a 5.000 empleados,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3723274171/...
4,Principal Data Architect,Caylent,Argentina (En remoto),De 201 a 500 empleados · Servicios y consultor...,Jornada completa · Intermedio,19/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3724563612/...
...,...,...,...,...,...,...,...,...
98,Agricultural Sales Talent Community,KWS Group,"Posadas, Misiones, Argentina (Híbrido)",De 5.001 a 10.000 empleados · Agricultura,Jornada completa · Algo de responsabilidad,19/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3724683949/...
99,"Senior Program Manager, South America",German Entrepreneurship,"Provincia de Buenos Aires, Argentina (Presencial)",De 51 a 200 empleados · Consultoría y servicio...,Jornada completa · Algo de responsabilidad,20/09/2023,[Gestión de proyectos],https://www.linkedin.com/jobs/view/3724745811/...
100,Data Scientist,Aerosapien Technologies ™,América Latina (En remoto),De 51 a 200 empleados · Fabricación de compone...,Jornada completa · Algo de responsabilidad,20/09/2023,"[Aprendizaje automático, Ciencia de datos, Min...",https://www.linkedin.com/jobs/view/3723224667/...
101,Business Intelligence/Financial Analyst,Plan A Technologies,América Latina (En remoto),De 201 a 500 empleados · Desarrollo de software,Jornada completa · Intermedio,19/09/2023,"[Microsoft Excel, Microsoft Power BI, Microsof...",https://www.linkedin.com/jobs/view/3724547497/...


Pasamos a la etapa de transformación general donde crearemos nuevas columnas, eliminaremos algunos y cambiaremos el tipo de datos de una columna específicamente

In [167]:
def functionWorkModality(word):
    coincidences = re.findall( r'\((.*?)\)', word)
    content = ' '.join(coincidences)
    return content if content else 'Dato no encontrado'

In [168]:
df['workModality'] = df['companyLocation'].apply(functionWorkModality) # Como en la columna "companyLocation" se encuentra la modalidad de empleo, lo que hacemos es crear una columna nueva con ese dato solo
                                                                                                

In [169]:
df

,titleJobs,comanyName,companyLocation,employeeRank,workType,dateJobs,skill,link,workModality
0,Mid. QA Automation Engineer - Buenos Aires,Devlane,Buenos Aires y alrededores (Híbrido),De 51 a 200 empleados,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3723261581/...,Híbrido
1,Data Scientist,Rocbird,"Córdoba, Córdoba, Argentina (Híbrido)",De 11 a 50 empleados,Jornada completa,20/09/2023,"[Análisis de datos, Ciencia de datos, Microsof...",https://www.linkedin.com/jobs/view/3723215447/...,Híbrido
2,Financial Analyst I - Pto. Madero,Worley,"Buenos Aires, Provincia de Buenos Aires, Argen...",Más de 10.001 empleados · Servicios profesionales,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3725055225/...,Presencial
3,Jefe de Nuevas Obras y Proyectos de Infraestru...,Clínica Universitaria Reina Fabiola,"Córdoba, Córdoba, Argentina (Presencial)",De 1.001 a 5.000 empleados,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3723274171/...,Presencial
4,Principal Data Architect,Caylent,Argentina (En remoto),De 201 a 500 empleados · Servicios y consultor...,Jornada completa · Intermedio,19/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3724563612/...,En remoto
...,...,...,...,...,...,...,...,...,...
98,Agricultural Sales Talent Community,KWS Group,"Posadas, Misiones, Argentina (Híbrido)",De 5.001 a 10.000 empleados · Agricultura,Jornada completa · Algo de responsabilidad,19/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3724683949/...,Híbrido
99,"Senior Program Manager, South America",German Entrepreneurship,"Provincia de Buenos Aires, Argentina (Presencial)",De 51 a 200 empleados · Consultoría y servicio...,Jornada completa · Algo de responsabilidad,20/09/2023,[Gestión de proyectos],https://www.linkedin.com/jobs/view/3724745811/...,Presencial
100,Data Scientist,Aerosapien Technologies ™,América Latina (En remoto),De 51 a 200 empleados · Fabricación de compone...,Jornada completa · Algo de responsabilidad,20/09/2023,"[Aprendizaje automático, Ciencia de datos, Min...",https://www.linkedin.com/jobs/view/3723224667/...,En remoto
101,Business Intelligence/Financial Analyst,Plan A Technologies,América Latina (En remoto),De 201 a 500 empleados · Desarrollo de software,Jornada completa · Intermedio,19/09/2023,"[Microsoft Excel, Microsoft Power BI, Microsof...",https://www.linkedin.com/jobs/view/3724547497/...,En remoto


In [170]:
def deleteParentheses(word):
    return re.sub(r'\([^)]*\)', '', word)


In [171]:
df['companyLocation']= df['companyLocation'].apply(deleteParentheses) # Ahora eliminamos la modalidad de empleo de la columna "companyLocation"

In [172]:
df

,titleJobs,comanyName,companyLocation,employeeRank,workType,dateJobs,skill,link,workModality
0,Mid. QA Automation Engineer - Buenos Aires,Devlane,Buenos Aires y alrededores,De 51 a 200 empleados,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3723261581/...,Híbrido
1,Data Scientist,Rocbird,"Córdoba, Córdoba, Argentina",De 11 a 50 empleados,Jornada completa,20/09/2023,"[Análisis de datos, Ciencia de datos, Microsof...",https://www.linkedin.com/jobs/view/3723215447/...,Híbrido
2,Financial Analyst I - Pto. Madero,Worley,"Buenos Aires, Provincia de Buenos Aires, Argen...",Más de 10.001 empleados · Servicios profesionales,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3725055225/...,Presencial
3,Jefe de Nuevas Obras y Proyectos de Infraestru...,Clínica Universitaria Reina Fabiola,"Córdoba, Córdoba, Argentina",De 1.001 a 5.000 empleados,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3723274171/...,Presencial
4,Principal Data Architect,Caylent,Argentina,De 201 a 500 empleados · Servicios y consultor...,Jornada completa · Intermedio,19/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3724563612/...,En remoto
...,...,...,...,...,...,...,...,...,...
98,Agricultural Sales Talent Community,KWS Group,"Posadas, Misiones, Argentina",De 5.001 a 10.000 empleados · Agricultura,Jornada completa · Algo de responsabilidad,19/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3724683949/...,Híbrido
99,"Senior Program Manager, South America",German Entrepreneurship,"Provincia de Buenos Aires, Argentina",De 51 a 200 empleados · Consultoría y servicio...,Jornada completa · Algo de responsabilidad,20/09/2023,[Gestión de proyectos],https://www.linkedin.com/jobs/view/3724745811/...,Presencial
100,Data Scientist,Aerosapien Technologies ™,América Latina,De 51 a 200 empleados · Fabricación de compone...,Jornada completa · Algo de responsabilidad,20/09/2023,"[Aprendizaje automático, Ciencia de datos, Min...",https://www.linkedin.com/jobs/view/3723224667/...,En remoto
101,Business Intelligence/Financial Analyst,Plan A Technologies,América Latina,De 201 a 500 empleados · Desarrollo de software,Jornada completa · Intermedio,19/09/2023,"[Microsoft Excel, Microsoft Power BI, Microsof...",https://www.linkedin.com/jobs/view/3724547497/...,En remoto


In [174]:
df['employeeRank'].value_counts()

employeeRank
De 1.001 a 5.000 empleados · Servicios y consultoría de TI                         19
Más de 10.001 empleados · Servicios y consultoría de TI                            15
Más de 10.001 empleados · Consultoría y servicios a empresas                       11
Más de 10.001 empleados · Servicios financieros                                     9
De 11 a 50 empleados · Tecnología, información e internet                           6
De 5.001 a 10.000 empleados · Servicios y consultoría de TI                         3
De 501 a 1.000 empleados · Tecnología, información e internet                       3
Más de 10.001 empleados · Fabricación de productos farmacéuticos                    3
Más de 10.001 empleados · Hospitales y atención sanitaria                           3
De 201 a 500 empleados · Desarrollo de software                                     2
Más de 10.001 empleados · Telecomunicaciones                                        2
De 51 a 200 empleados · Servicios y consu

In [175]:
def sector(word):
    lista = word.split('·')
    if len(lista) == 2:
        return lista[1]
    else: 
        return 'Dato no encontrado'



In [176]:
len('De 501 a 1.000 empleados · Servicios de TI y consultoría de TI'.split('·')) # ejemplo

2

In [177]:
df['industry']= df['employeeRank'].apply(sector) # Separamos industria de la columna rango de empleados creando "industry"

In [178]:
df.head()

,titleJobs,comanyName,companyLocation,employeeRank,workType,dateJobs,skill,link,workModality,industry
0,Mid. QA Automation Engineer - Buenos Aires,Devlane,Buenos Aires y alrededores,De 51 a 200 empleados,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3723261581/...,Híbrido,Dato no encontrado
1,Data Scientist,Rocbird,"Córdoba, Córdoba, Argentina",De 11 a 50 empleados,Jornada completa,20/09/2023,"[Análisis de datos, Ciencia de datos, Microsof...",https://www.linkedin.com/jobs/view/3723215447/...,Híbrido,Dato no encontrado
2,Financial Analyst I - Pto. Madero,Worley,"Buenos Aires, Provincia de Buenos Aires, Argen...",Más de 10.001 empleados · Servicios profesionales,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3725055225/...,Presencial,Servicios profesionales
3,Jefe de Nuevas Obras y Proyectos de Infraestru...,Clínica Universitaria Reina Fabiola,"Córdoba, Córdoba, Argentina",De 1.001 a 5.000 empleados,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3723274171/...,Presencial,Dato no encontrado
4,Principal Data Architect,Caylent,Argentina,De 201 a 500 empleados · Servicios y consultor...,Jornada completa · Intermedio,19/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3724563612/...,En remoto,Servicios y consultoría de TI


In [179]:
def employeeRankFunction(word):
    lista = word.split('·')
    return lista[0]

In [180]:
df['employeeRank']= df['employeeRank'].apply(employeeRankFunction) # Ahora eliminamos la parte de industria de la columna rango de empleados

In [181]:
df.head()

,titleJobs,comanyName,companyLocation,employeeRank,workType,dateJobs,skill,link,workModality,industry
0,Mid. QA Automation Engineer - Buenos Aires,Devlane,Buenos Aires y alrededores,De 51 a 200 empleados,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3723261581/...,Híbrido,Dato no encontrado
1,Data Scientist,Rocbird,"Córdoba, Córdoba, Argentina",De 11 a 50 empleados,Jornada completa,20/09/2023,"[Análisis de datos, Ciencia de datos, Microsof...",https://www.linkedin.com/jobs/view/3723215447/...,Híbrido,Dato no encontrado
2,Financial Analyst I - Pto. Madero,Worley,"Buenos Aires, Provincia de Buenos Aires, Argen...",Más de 10.001 empleados,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3725055225/...,Presencial,Servicios profesionales
3,Jefe de Nuevas Obras y Proyectos de Infraestru...,Clínica Universitaria Reina Fabiola,"Córdoba, Córdoba, Argentina",De 1.001 a 5.000 empleados,Jornada completa,20/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3723274171/...,Presencial,Dato no encontrado
4,Principal Data Architect,Caylent,Argentina,De 201 a 500 empleados,Jornada completa · Intermedio,19/09/2023,Dato no encontrado,https://www.linkedin.com/jobs/view/3724563612/...,En remoto,Servicios y consultoría de TI


In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titleJobs        103 non-null    object
 1   comanyName       103 non-null    object
 2   companyLocation  103 non-null    object
 3   employeeRank     103 non-null    object
 4   workType         103 non-null    object
 5   dateJobs         103 non-null    object
 6   skill            103 non-null    object
 7   link             103 non-null    object
 8   workModality     103 non-null    object
 9   industry         103 non-null    object
dtypes: object(10)
memory usage: 8.2+ KB


In [183]:
df['date'] = pd.to_datetime(df['dateJobs']) # Convertimos el tipo de datos de "dateJobs" a fecha

C:\Users\julie\AppData\Local\Temp\ipykernel_20612\1394614003.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['dateJobs'])


In [184]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   titleJobs        103 non-null    object        
 1   comanyName       103 non-null    object        
 2   companyLocation  103 non-null    object        
 3   employeeRank     103 non-null    object        
 4   workType         103 non-null    object        
 5   dateJobs         103 non-null    object        
 6   skill            103 non-null    object        
 7   link             103 non-null    object        
 8   workModality     103 non-null    object        
 9   industry         103 non-null    object        
 10  date             103 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(10)
memory usage: 9.0+ KB


In [186]:
df.drop('dateJobs', axis=1,inplace=True) # Eliminamos "dateJobs" para quedarnos solo con "date"

In [187]:
df

,titleJobs,comanyName,companyLocation,employeeRank,workType,skill,link,workModality,industry,date
0,Mid. QA Automation Engineer - Buenos Aires,Devlane,Buenos Aires y alrededores,De 51 a 200 empleados,Jornada completa,Dato no encontrado,https://www.linkedin.com/jobs/view/3723261581/...,Híbrido,Dato no encontrado,2023-09-20
1,Data Scientist,Rocbird,"Córdoba, Córdoba, Argentina",De 11 a 50 empleados,Jornada completa,"[Análisis de datos, Ciencia de datos, Microsof...",https://www.linkedin.com/jobs/view/3723215447/...,Híbrido,Dato no encontrado,2023-09-20
2,Financial Analyst I - Pto. Madero,Worley,"Buenos Aires, Provincia de Buenos Aires, Argen...",Más de 10.001 empleados,Jornada completa,Dato no encontrado,https://www.linkedin.com/jobs/view/3725055225/...,Presencial,Servicios profesionales,2023-09-20
3,Jefe de Nuevas Obras y Proyectos de Infraestru...,Clínica Universitaria Reina Fabiola,"Córdoba, Córdoba, Argentina",De 1.001 a 5.000 empleados,Jornada completa,Dato no encontrado,https://www.linkedin.com/jobs/view/3723274171/...,Presencial,Dato no encontrado,2023-09-20
4,Principal Data Architect,Caylent,Argentina,De 201 a 500 empleados,Jornada completa · Intermedio,Dato no encontrado,https://www.linkedin.com/jobs/view/3724563612/...,En remoto,Servicios y consultoría de TI,2023-09-19
...,...,...,...,...,...,...,...,...,...,...
98,Agricultural Sales Talent Community,KWS Group,"Posadas, Misiones, Argentina",De 5.001 a 10.000 empleados,Jornada completa · Algo de responsabilidad,Dato no encontrado,https://www.linkedin.com/jobs/view/3724683949/...,Híbrido,Agricultura,2023-09-19
99,"Senior Program Manager, South America",German Entrepreneurship,"Provincia de Buenos Aires, Argentina",De 51 a 200 empleados,Jornada completa · Algo de responsabilidad,[Gestión de proyectos],https://www.linkedin.com/jobs/view/3724745811/...,Presencial,Consultoría y servicios a empresas,2023-09-20
100,Data Scientist,Aerosapien Technologies ™,América Latina,De 51 a 200 empleados,Jornada completa · Algo de responsabilidad,"[Aprendizaje automático, Ciencia de datos, Min...",https://www.linkedin.com/jobs/view/3723224667/...,En remoto,Fabricación de componentes aeroespaciales y d...,2023-09-20
101,Business Intelligence/Financial Analyst,Plan A Technologies,América Latina,De 201 a 500 empleados,Jornada completa · Intermedio,"[Microsoft Excel, Microsoft Power BI, Microsof...",https://www.linkedin.com/jobs/view/3724547497/...,En remoto,Desarrollo de software,2023-09-19


Uniremos este DataFrame con el CSV que extrajimos anteriormente.

In [188]:
dfanterior = pd.read_csv('jobsdata.csv')

In [196]:
df.head(2)

,titleJobs,comanyName,companyLocation,employeeRank,workType,skill,link,workModality,industry,date
0,Mid. QA Automation Engineer - Buenos Aires,Devlane,Buenos Aires y alrededores,De 51 a 200 empleados,Jornada completa,Dato no encontrado,https://www.linkedin.com/jobs/view/3723261581/...,Híbrido,Dato no encontrado,2023-09-20
1,Data Scientist,Rocbird,"Córdoba, Córdoba, Argentina",De 11 a 50 empleados,Jornada completa,"[Análisis de datos, Ciencia de datos, Microsof...",https://www.linkedin.com/jobs/view/3723215447/...,Híbrido,Dato no encontrado,2023-09-20


In [197]:
dfanterior.head(2)

,titleJobs,comanyName,companyLocation,employeeRank,workType,skill,link,workModality,industry,date
0,Data Scientist,TinyBytes,Argentina,De 11 a 50 empleados,Jornada completa,"['Aprendizaje automático', 'Ciencia de datos',...",https://www.linkedin.com/jobs/view/3719640681/...,En remoto,Dato no encontrado,2023-09-18
1,Gerente de operaciones,La Fonte D'Oro,Buenos Aires y alrededores,De 201 a 500 empleados,Jornada completa,"['Analítica de datos', 'Aptitudes para la supe...",https://www.linkedin.com/jobs/view/3715813639/...,Presencial,Dato no encontrado,2023-09-18


In [190]:
df_resultado = pd.concat([df, dfanterior])

In [193]:
df_resultado

,titleJobs,comanyName,companyLocation,employeeRank,workType,skill,link,workModality,industry,date
0,Mid. QA Automation Engineer - Buenos Aires,Devlane,Buenos Aires y alrededores,De 51 a 200 empleados,Jornada completa,Dato no encontrado,https://www.linkedin.com/jobs/view/3723261581/...,Híbrido,Dato no encontrado,2023-09-20 00:00:00
1,Data Scientist,Rocbird,"Córdoba, Córdoba, Argentina",De 11 a 50 empleados,Jornada completa,"[Análisis de datos, Ciencia de datos, Microsof...",https://www.linkedin.com/jobs/view/3723215447/...,Híbrido,Dato no encontrado,2023-09-20 00:00:00
2,Financial Analyst I - Pto. Madero,Worley,"Buenos Aires, Provincia de Buenos Aires, Argen...",Más de 10.001 empleados,Jornada completa,Dato no encontrado,https://www.linkedin.com/jobs/view/3725055225/...,Presencial,Servicios profesionales,2023-09-20 00:00:00
3,Jefe de Nuevas Obras y Proyectos de Infraestru...,Clínica Universitaria Reina Fabiola,"Córdoba, Córdoba, Argentina",De 1.001 a 5.000 empleados,Jornada completa,Dato no encontrado,https://www.linkedin.com/jobs/view/3723274171/...,Presencial,Dato no encontrado,2023-09-20 00:00:00
4,Principal Data Architect,Caylent,Argentina,De 201 a 500 empleados,Jornada completa · Intermedio,Dato no encontrado,https://www.linkedin.com/jobs/view/3724563612/...,En remoto,Servicios y consultoría de TI,2023-09-19 00:00:00
...,...,...,...,...,...,...,...,...,...,...
25,Senior DevOps Engineer - Azure,Clevertech,Argentina,De 501 a 1.000 empleados,Jornada completa · Intermedio,"['Linux', 'Python', 'Amazon RDS', 'Bash', 'Ela...",https://www.linkedin.com/jobs/view/3723719193/...,En remoto,Servicios de TI y consultoría de TI,2023-09-18
26,iOS Developer,Clevertech,Argentina,De 501 a 1.000 empleados,Jornada completa · Intermedio,"['Algoritmos', 'Metodologías ágiles', 'Aplicac...",https://www.linkedin.com/jobs/view/3723694504/...,En remoto,Servicios de TI y consultoría de TI,2023-09-18
27,Java & Angular developer,IBM,"Buenos Aires, Provincia de Buenos Aires, Argen...",Más de 10.001 empleados,Jornada completa,Dato no encontrado,https://www.linkedin.com/jobs/view/3696802649/...,Presencial,Servicios de TI y consultoría de TI,2023-09-18
28,iOS Developer,Clevertech,"Córdoba y alrededores, Argentina",De 501 a 1.000 empleados,Jornada completa · Intermedio,"['Algoritmos', 'Metodologías ágiles', 'Aplicac...",https://www.linkedin.com/jobs/view/3723687998/...,En remoto,Servicios de TI y consultoría de TI,2023-09-18


In [194]:
df_resultado.to_csv('linkedinJobs.csv', index=False) # Finalmente lo guardamos en un archivo CSV